# Getting Started with the NeMo Agent Toolkit

In this notebook, we walk through the basics of using the toolkit, from installation all the way to creating and running your very own custom workflow.

## Environment Setup

Ensure you meet the following prerequisites:
1. Git
2. [uv](https://docs.astral.sh/uv/getting-started/installation/)
3. NeMo-Agent-Toolkit installed from source following [these instructions](https://github.com/cdgamarose-nv/NeMo-Agent-Toolkit/tree/develop?tab=readme-ov-file#install-from-source)

> **_NOTE:_**  Ensure you are running this notebook from the root directory of the cloned NeMo-Agent-Toolkit repository.

### Set API keys

In [13]:
import getpass
import os

if "NVIDIA_API_KEY" not in os.environ:
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key

if "TAVILY_API_KEY" not in os.environ:
    tavily_api_key = getpass.getpass("Enter your Tavily API key: ")
    os.environ["TAVILY_API_KEY"] = tavily_api_key

if "OPENAI_API_KEY" not in os.environ:
    openai_api_key = getpass.getpass("Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

## Bringing an Agent into the NeMo-Agent-Toolkit

NeMo Agent toolkit works side-by-side and complements any existing agentic framework or memory tool you're using and isn't tied to any specific agentic framework, long-term memory, or data source. This allows you to use your current technology stack - such as LangChain, LlamaIndex, CrewAI, and Microsoft Semantic Kernel, as well as customer enterprise frameworks and simple Python agents - without replatforming.

We'll walk you through how to achieve this.

To demonstrate this, let's say that you have the following simple langchain agent that answers generic user queries about current events by performing a web search using Tavily. We will show you how to bring this agent into the NeMo-Agent-Toolkit and benefit from the configurability, resuability, and easy user experience.

Run the following two cells to create the langchain agent that gets written to `.tmp/notebooks/examples/langchain_agent.py` and run it with an example input.

In [17]:
%%bash
mkdir -p .tmp/notebooks/examples/
cat << EOF > .tmp/notebooks/examples/langchain_agent.py
import os
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize a tool to search the web
tavily_kwargs = {"max_results": 2, "api_key": os.getenv("TAVILY_API_KEY")}
search = TavilySearchResults(**tavily_kwargs)

# Create a list of tools for the agent
tools = [search]

# Initialize a LLM client
llm_kwargs = {
    "model_name": "meta/llama-3.3-70b-instruct",
    "temperature": 0.0,
    "max_tokens": 1024,
    "api_key": os.getenv("NVIDIA_API_KEY"),
}
llm = ChatNVIDIA(**llm_kwargs)

# Use an open source prompt
prompt = hub.pull("hwchase17/react-chat")

# Initialize a ReAct agent
react_agent = create_react_agent(llm=llm, tools=tools, prompt=prompt, stop_sequence=["\nObservation"])

# Initialize an agent executor to iterate through reasoning steps
agent_executor = AgentExecutor(agent=react_agent,
                               tools=tools,
                               max_iterations=15,
                               handle_parsing_errors=True,
                               verbose=True)

# Invoke the agent with a user query
response = agent_executor.invoke({"input": "Who is the current Pope?", "chat_history": []})

# Print the response
print(response["output"])
EOF

In [18]:
!python .tmp/notebooks/examples/langchain_agent.py

/localhome/local-cdgamarose/NeMo-Agent-Toolkit/.tmp/notebooks/examples/langchain_agent.py:10: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(**tavily_kwargs)
/localhome/local-cdgamarose/NeMo-Agent-Toolkit/.venv/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: tavily_search_results_json
Action Input: "Who is the current pope?"
Observ[{'title': 'Who is the new Pope Leo XIV and what is his background? - NPR', 'url': 'https://www.npr.org/2025/05/08/g-s1-65147/ne

#### Creating a new NeMo-Agent-Toolkit Workflow 

Bringing this agent into the toolkit requires creating a new workflow and configuring the tools, and so on. A workflow is a self-contained pipeline that orchestrates tools (e.g., custom arithmetic tools, web search, RAG) and one or more LLMs to process user inputs and generate outputs.

With our `aiq create` sub-command, you can scaffold and register new workflows within seconds. 


In [15]:
!aiq workflow create --workflow-dir .tmp/notebooks/examples my_agent_workflow

Installing workflow 'my_agent_workflow'...
Workflow 'my_agent_workflow' installed successfully.
Workflow 'my_agent_workflow' created successfully in '/localhome/local-cdgamarose/NeMo-Agent-Toolkit/.tmp/notebooks/examples/my_agent_workflow'.


This command:
- Creates a new directory examples/my_agent_workflow.
- Sets up the necessary files and folders.
- Installs the new Python package for your workflow.

The registration process is built around two main components:
- A configuration class that inherits from WorkflowBaseConfig.
- A decorated async function (with @register_workflow) that yields a callable “response function.”
Once configured, you can run workflows via the command line (aiq run) or launch them as services (aiq serve) to handle requests in real time.

#### Customizing your Workflow

Now its time to define the same langchain agent inside your newly created workflow. This is as simple as making a few code additions to the `my_agent_workflow_function`.
- Add langchain framework wrappers (all this does is indicate which framework you are wrapping your code in which enables profiling the workflow later)
- Paste your agent initialization code inside the `my_agent_workflow_function`
- Paste your agent invocation code inside the `_response_fn` function

Your final `my_agent_workflow_function.py` should look like:
```python
import logging

from pydantic import Field

from aiq.builder.builder import Builder
from aiq.builder.function_info import FunctionInfo
from aiq.cli.register_workflow import register_function
from aiq.data_models.function import FunctionBaseConfig
from aiq.builder.framework_enum import LLMFrameworkEnum

logger = logging.getLogger(__name__)


class MyAgentWorkflowFunctionConfig(FunctionBaseConfig, name="my_agent_workflow"):
    """
    AIQ Toolkit function template. Please update the description.
    """
    parameter: str = Field(default="default_value", description="Notional description for this parameter")


@register_function(config_type=MyAgentWorkflowFunctionConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def my_agent_workflow_function(
    config: MyAgentWorkflowFunctionConfig, builder: Builder
):
    import os
    from langchain import hub
    from langchain.agents import AgentExecutor
    from langchain.agents import create_react_agent
    from langchain_nvidia_ai_endpoints import ChatNVIDIA
    from langchain_community.tools.tavily_search import TavilySearchResults

    # Initialize a tool to search the web
    tavily_kwargs = {"max_results": 2, "api_key": os.getenv("TAVILY_API_KEY")}
    search = TavilySearchResults(**tavily_kwargs)

    # Create a list of tools for the agent
    tools = [search]

    # Initialize a LLM client
    llm_kwargs = {
        "model_name": "meta/llama-3.3-70b-instruct",
        "temperature": 0.0,
        "max_tokens": 1024,
        "api_key": os.getenv("NVIDIA_API_KEY"),
    }
    llm = ChatNVIDIA(**llm_kwargs)

    # Use an open source prompt
    prompt = hub.pull("hwchase17/react-chat")

    # Initialize a ReAct agent
    react_agent = create_react_agent(llm=llm, tools=tools, prompt=prompt, stop_sequence=["\nObservation"])

    # Initialize an agent executor to iterate through reasoning steps
    agent_executor = AgentExecutor(agent=react_agent,
                                tools=tools,
                                max_iterations=15,
                                handle_parsing_errors=True,
                                verbose=True)    

    async def _response_fn(input_message: str) -> str:
        response = agent_executor.invoke({"input": input_message, "chat_history": []})

        return response["output"]

    try:
        yield FunctionInfo.create(single_fn=_response_fn)
    except GeneratorExit:
        print("Function exited early!")
    finally:
        print("Cleaning up my_agent_workflow workflow.")

```

Once you have your workflow registered, you can reference it by its `_type` in a YAML file. 

For example:

```yaml
workflow:
  _type: my_agent_workflow
```

With this config, the toolkit knows to:
- Load the workflow named "my_agent_workflow".
- Call the `my_agent_workflow` function so it can yield the `_response_fn`.

#### Running your Workflow

In order to test your new agent using the `aiq` CLI, run:

In [20]:
!aiq run --config_file .tmp/notebooks/examples/my_agent_workflow/configs/config.yml --input "Who is the current Pope?"

2025-07-17 17:33:07,663 - aiq.runtime.loader - WARNING - Loading module 'aiq_profiler_agent.register' from entry point 'aiq_profiler_agent' took a long time (410.322428 ms). Ensure all imports are inside your registered functions.
2025-07-17 17:33:08,338 - aiq.runtime.loader - WARNING - Loading module 'aiq_alert_triage_agent.register' from entry point 'aiq_alert_triage_agent' took a long time (475.420713 ms). Ensure all imports are inside your registered functions.
2025-07-17 17:33:08,521 - aiq.cli.commands.start - INFO - Starting AIQ Toolkit from config file: 'notebooks/examples/my_agent_workflow/configs/config.yml'
/localhome/local-cdgamarose/NeMo-Agent-Toolkit/notebooks/examples/my_agent_workflow/src/my_agent_workflow/my_agent_workflow_function.py:34: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To 

As shown above, this will return the same output as your previously created langchain agent.

#### Runtime Configurations

To benefit from the configurability of this toolkit, we can update the configuration object and config file along with the function to use the parameters at runtime.

This involves allowing the toolkit to sets up your tools, LLM, and any additional logic like maximum number of historical messages to provide to the agent, maximum number of iterations to run the agent, description of the agent and so on.

The toolkit will make use of the `Builder` class to utilize them at runtime.

Your final configuration object should look like this:
```python
class MyAgentWorkflowFunctionConfig(FunctionBaseConfig, name="my_agent_workflow"):
    """
    AIQ Toolkit function template. Please update the description.
    """
    tool_names: list[FunctionRef] = Field(default=[], description="List of tool names to use")
    llm_name: LLMRef = Field(description="LLM name to use")
    max_history: int = Field(default=10, description="Maximum number of historical messages to provide to the agent")
    max_iterations: int = Field(default=15, description="Maximum number of iterations to run the agent")
    handle_parsing_errors: bool = Field(default=True, description="Whether to handle parsing errors")
    verbose: bool = Field(default=True, description="Whether to print verbose output")
    description: str = Field(default="", description="Description of the agent")
```

You can then replace:
```python
tool = [search]
```
with 
```python
tools = builder.get_tools(config.tool_names, wrapper_type=LLMFrameworkEnum.LANGCHAIN)
```
> **Note**: This allows you to bring in tools from other frameworks like llama index as well and wrap them with langchain since you are implementing your agent in langchain.

In a similar way, you can initialize your llm by utilizing the parameters from the configuration object in the following way:
```python
llm = await builder.get_llm(config.llm_name, wrapper_type=LLMFrameworkEnum.LANGCHAIN)
```

For each tool or reusable plugin, there are potentially multiple optional parameters with default values that can be overridden. The `aiq info components` command can be used to list all available parameters. For example, to list all available parameters for the LLM nim type run:

```bash
aiq info components -t llm_provider -q nim
```

#### Reusing the Inbuilt Tavily Search Function

We can also make use of some of many example functions that the toolkit provides for common use cases. In this agent example, rather than reimplementing the tavily search, we will use the inbuilt function for internet search which is built on top of langchain's tavily search API. You can list available functions using the following:

In [22]:
!aiq info components -t function -q tavily_internet_search

2025-07-17 17:36:47,501 - aiq.runtime.loader - WARNING - Loading module 'aiq_profiler_agent.register' from entry point 'aiq_profiler_agent' took a long time (413.591385 ms). Ensure all imports are inside your registered functions.
2025-07-17 17:36:48,200 - aiq.runtime.loader - WARNING - Loading module 'aiq_alert_triage_agent.register' from entry point 'aiq_alert_triage_agent' took a long time (499.726057 ms). Ensure all imports are inside your registered functions.
                           AIQ Toolkit Search Results                           
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ package    ┃ version        ┃ component_ty… ┃ component_name ┃ description   ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ aiqtoolkit │ 0.1.dev225+g54 │ function      │ tavily_interne │ Tool that     │
│            │ 0408c.d2025071 │               │ t_search       │ retrieves     │
│            │ 7              │             

This function can be used any number of times in the configuration YAML by specifying the `_type` as `tavily_internet_search`

```yaml
functions:
  my_internet_search:
    _type: tavily_internet_search
    max_results: 2
    api_key: $TAVILY_API_KEY
```

#### Final Code and Configuration
The final code for your workflow can be found in [this example](notebooks/examples/my_agent_workflow/src/my_agent_workflow/my_agent_workflow_function.py)

In [ ]:
%%bash

# Modify the config file
cat <<EOF > .tmp/notebooks/examples/my_agent_workflow/configs/config_modified.yml
general:
  use_uvloop: true
  logging:
    console:
      _type: console
      level: WARN

llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.3-70b-instruct
    temperature: 0.0
    max_tokens: 1024
    api_key: $NVIDIA_API_KEY

functions:
  my_internet_search:
    _type: tavily_internet_search
    max_results: 2
    api_key: $TAVILY_API_KEY

workflow:
  _type: my_agent_workflow
  tool_names:
    - my_internet_search
  llm_name: nim_llm
  max_history: 10
  max_iterations: 15
  description: "A helpful assistant that can search the internet for information"

EOF

In [26]:
!aiq run --config_file .tmp/notebooks/examples/my_agent_workflow/configs/config_modified.yml --input "Who is the current Pope?"

2025-07-17 17:38:25,010 - aiq.runtime.loader - WARNING - Loading module 'aiq_profiler_agent.register' from entry point 'aiq_profiler_agent' took a long time (422.947645 ms). Ensure all imports are inside your registered functions.
2025-07-17 17:38:25,713 - aiq.runtime.loader - WARNING - Loading module 'aiq_alert_triage_agent.register' from entry point 'aiq_alert_triage_agent' took a long time (504.836559 ms). Ensure all imports are inside your registered functions.
2025-07-17 17:38:25,903 - aiq.cli.commands.start - INFO - Starting AIQ Toolkit from config file: 'notebooks/examples/my_agent_workflow/configs/config_modified.yml'
2025-07-17 17:38:25,906 - aiq.cli.commands.start - WARNING - The front end type in the config file (fastapi) does not match the command name (console). Overwriting the config file front end.
/localhome/local-cdgamarose/NeMo-Agent-Toolkit/notebooks/examples/my_agent_workflow/src/my_agent_workflow/my_agent_workflow_function.py:34: LangChainDeprecationWarning: The cl

#### AIQ Serve

You can also use the `aiq serve` sub-command to launch a server and make HTTP requests to the endpoints as shown below.

In [ ]:
%%bash
aiq serve --config_file=notebooks/examples/my_agent_workflow/configs/config_modified.yml

In [ ]:
%%bash
curl --request POST \
  --url http://localhost:8000/chat/stream \
  --header 'Content-Type: application/json' \
  --data '{
    "messages": [
      {
        "role": "user",
        "content": "Who is the current Pope?"
      }
    ]
}'
pkill -9 aiq

#### Reusing the Inbuilt ReAct Agent

NeMo Agent Toolkit has a reusable react agent function. We can reuse that agent here to simplify the workflow even further.

In [32]:
!aiq info components -t function -q aqitoolkit.react_agent

2025-07-17 17:51:14,742 - aiq.runtime.loader - WARNING - Loading module 'aiq_profiler_agent.register' from entry point 'aiq_profiler_agent' took a long time (409.621716 ms). Ensure all imports are inside your registered functions.
2025-07-17 17:51:15,423 - aiq.runtime.loader - WARNING - Loading module 'aiq_alert_triage_agent.register' from entry point 'aiq_alert_triage_agent' took a long time (482.413769 ms). Ensure all imports are inside your registered functions.
                           AIQ Toolkit Search Results                           
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ package       ┃ version       ┃ component_ty… ┃ component_n… ┃ description   ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ aiq_simple_ca │ 0.1.dev225+g5 │ function      │ retry_react_ │ This function │
│ lculator_hitl │ 40408c.d20250 │               │ agent        │ creates a     │
│               │ 717           │           

In [34]:
%%bash

# Modify the config file
cat <<EOF > notebooks/examples/my_agent_workflow/configs/config_modified.yml
general:
  use_uvloop: true
  logging:
    console:
      _type: console
      level: WARN

llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.3-70b-instruct
    temperature: 0.0
    max_tokens: 1024
    api_key: $NVIDIA_API_KEY

functions:
  my_internet_search:
    _type: tavily_internet_search
    max_results: 2
    api_key: $TAVILY_API_KEY

workflow:
  _type: react_agent
  tool_names:
    - my_internet_search
  llm_name: nim_llm
  max_history: 10
  max_iterations: 15
  description: "A helpful assistant that can search the internet for information"

EOF

In [35]:
!aiq run --config_file notebooks/examples/my_agent_workflow/configs/config_modified.yml --input "Who is the current Pope?"

2025-07-17 17:55:49,625 - aiq.runtime.loader - WARNING - Loading module 'aiq_profiler_agent.register' from entry point 'aiq_profiler_agent' took a long time (418.840885 ms). Ensure all imports are inside your registered functions.
2025-07-17 17:55:50,300 - aiq.runtime.loader - WARNING - Loading module 'aiq_alert_triage_agent.register' from entry point 'aiq_alert_triage_agent' took a long time (478.560925 ms). Ensure all imports are inside your registered functions.
2025-07-17 17:55:50,481 - aiq.cli.commands.start - INFO - Starting AIQ Toolkit from config file: 'notebooks/examples/my_agent_workflow/configs/config_modified.yml'
2025-07-17 17:55:50,483 - aiq.cli.commands.start - WARNING - The front end type in the config file (fastapi) does not match the command name (console). Overwriting the config file front end.

Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 1
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Retriever